In [1]:
pip install web3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 501 kB 7.1 MB/s 
     |████████████████████████████████| 82 kB 1.1 MB/s 
     |████████████████████████████████| 102 kB 15.3 MB/s 
     |████████████████████████████████| 101 kB 12.9 MB/s 
     |████████████████████████████████| 1.1 MB 56.7 MB/s 
     |████████████████████████████████| 45 kB 3.4 MB/s 
     |████████████████████████████████| 241 kB 44.6 MB/s 
     |████████████████████████████████| 2.3 MB 58.1 MB/s 
     |████████████████████████████████| 1.8 MB 53.2 MB/s 
     |████████████████████████████████| 1.9 MB 65.1 MB/s 
  Created wheel for parsimonious: filename=parsimonious-0.8.1-py3-none-any.whl size=42725 sha256=f1a5f5a2899e2104317c7b020ba204c3121b786d1e594f348699bdc526bfb5b3
  Stored in directory: /root/.cache/pip/wheels/d8/af/19/fb896f509a437aca2dcf62583e84d7fb2cd5b628c1564a609c
  Created wheel for varint: filename=varint-1.0.2-py3-non

In [1]:
from google.colab import drive, files
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
pip install pickle5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 132 kB 6.6 MB/s 
  Created wheel for pickle5: filename=pickle5-0.0.11-cp38-cp38-linux_x86_64.whl size=236285 sha256=3bf8ce247f485ce419c4b4449fb23a4c3f19e2191928e689543c9b9f2b223cb9
  Stored in directory: /root/.cache/pip/wheels/25/d4/61/dbd8edd1a0d656be7b4267c85db3b61951eb60016a0154a122
Successfully built pickle5


In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split #to split the dataset for training and testing
from sklearn.neural_network import MLPClassifier #Importing MLPClassifier for classification.
from sklearn import metrics
import json
from web3 import Web3
import os
import datetime;
from datetime import datetime
# Save Model Using joblib
import pickle 

In [13]:
# add your blockchain connection information
infura_url = 'https://goerli.infura.io/v3/5ff13871021244b79ad9642b2f6499e3'
web3 = Web3(Web3.HTTPProvider(infura_url))
chain_id = 5

account = "0xc144cD60Be02F5d5C6CFfcb56DcE32D99097Afb9"
private_key = '3cd656d14571c2f3c39b97dc273060f7653ec6c2fbbb916e717b4e9f1e28f147'

#contract address and abi
contract_Address = '0x8e8EF6DbFF63d25e73166c8225f1b0aDafBF255B'
contract_abi = json.loads('[ 	{ 		"inputs": [ 			{ 				"internalType": "string", 				"name": "_name", 				"type": "string" 			}, 			{ 				"internalType": "string", 				"name": "_hash", 				"type": "string" 			} 		], 		"name": "storeModel", 		"outputs": [], 		"stateMutability": "nonpayable", 		"type": "function" 	}, 	{ 		"inputs": [ 			{ 				"internalType": "string", 				"name": "_prediciton", 				"type": "string" 			}, 			{ 				"internalType": "string", 				"name": "_timeData", 				"type": "string" 			} 		], 		"name": "storePrediction", 		"outputs": [], 		"stateMutability": "nonpayable", 		"type": "function" 	}, 	{ 		"inputs": [ 			{ 				"internalType": "string[]", 				"name": "_records", 				"type": "string[]" 			} 		], 		"name": "storeRecords", 		"outputs": [], 		"stateMutability": "nonpayable", 		"type": "function" 	}, 	{ 		"inputs": [], 		"name": "retrieveModels", 		"outputs": [ 			{ 				"components": [ 					{ 						"internalType": "string", 						"name": "name", 						"type": "string" 					}, 					{ 						"internalType": "string", 						"name": "hash", 						"type": "string" 					} 				], 				"internalType": "struct IA.Model[]", 				"name": "", 				"type": "tuple[]" 			} 		], 		"stateMutability": "view", 		"type": "function" 	}, 	{ 		"inputs": [], 		"name": "retrievePredicitons", 		"outputs": [ 			{ 				"components": [ 					{ 						"internalType": "string", 						"name": "prediction", 						"type": "string" 					}, 					{ 						"internalType": "string", 						"name": "timeData", 						"type": "string" 					} 				], 				"internalType": "struct IA.Prediction[]", 				"name": "", 				"type": "tuple[]" 			} 		], 		"stateMutability": "view", 		"type": "function" 	}, 	{ 		"inputs": [], 		"name": "retrieveRecords", 		"outputs": [ 			{ 				"internalType": "string[]", 				"name": "", 				"type": "string[]" 			} 		], 		"stateMutability": "view", 		"type": "function" 	} ]')

contract = web3.eth.contract(address=contract_Address, abi=contract_abi)
nonce = web3.eth.getTransactionCount(account)

In [14]:
def obterModelos():
    # Wait for transaction to be mined
    transaction = contract.functions.retrieveModels().call()
    
    return transaction

In [15]:
def enviarPredicao(_predicao, _timestamp):   
    transaction = contract.functions.storePrediction(_predicao, _timestamp).buildTransaction(
        {
            "gasPrice": web3.eth.gas_price,
            "chainId": chain_id,
            "from": account,
            "nonce": nonce 
        }
    )
    signed_transaction = web3.eth.account.sign_transaction(transaction, private_key = private_key)
    transaction_hash = web3.eth.send_raw_transaction(signed_transaction.rawTransaction)
    transaction_receipt = web3.eth.wait_for_transaction_receipt(transaction_hash)

In [16]:
def compare_strings(str1, str2):
    count1 = 0
    count2 = 0
      
    for i in range(len(str1)):
        if str1[i] >= "0" and str1[i] <= "9":
            count1 += 1
      
    for i in range(len(str2)):
        if str2[i] >= "0" and str2[i] <= "9":
            count2 += 1
      
    return count1 == count2


def obterModelo(name):
  for model in obterModelos():    
    if compare_strings(model[0], name): 
      return model[1]   
     

print(obterModelo('IrisModel11'))

QmX1z3MKF1nrnJoKHvbMETHyETBPYeW7GK2Xz6ByjGLwM2


In [17]:
pip install requests

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [18]:
# Importe a biblioteca
import requests

api_key = "9f6052849e81cca8eee4"
secret_api_key = "1c35ddeb9ed231e7e610647f8d13bb74f52cf35bc36e3cc17ed96db5c9069534"

model_hash = obterModelo('IrisModel11')

url = f'https://gateway.pinata.cloud/ipfs/{model_hash}'
headers = {'pinata_api_key': api_key, 'pinata_secret_api_key': secret_api_key}
response = requests.get(url, headers=headers)

# Verifique se a solicitação foi bem-sucedida
if response.status_code == 200:
  with open('modelo.h5', 'wb') as f:
    f.write(response.content)
else:
  print('Erro ao fazer o download do modelo:', response.text)


loaded_model = pickle.load(open('modelo.h5', 'rb'))

# new data to be classified
#X_new = np.array([[5.1,3.5,1.4,0.2]])
X_new = np.array([[1.1,2.5,2.4,1.2]])

prediction = loaded_model.predict(X_new) 
print(prediction)

current_time = datetime.now()  
time_stamp = current_time.timestamp()
date_time = datetime.fromtimestamp(time_stamp)
print("The date and time is:", date_time)

enviarPredicao(str(prediction), str(date_time))

/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(


['Iris-versicolor']
The date and time is: 2022-12-31 05:09:59.244034
